In [2]:
!pip install pandas
!pip install scikit-surprise


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from surprise import KNNWithMeans
from surprise import SVD, SVDpp
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

import time

In [4]:
start_time = time.time()
df = pd.read_csv("OrderHistory.csv")
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

Done in 0.503s


In [5]:
df

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons
...,...,...,...,...,...,...,...,...,...
229994,229995,29370,A2MKDOEB6K6KS1,5,16,0,Haryana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229995,229996,29370,A3DZMDTBJ7PQX7,5,19,1,Himachal Pradesh,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229996,229997,29370,A2TNGO6SGRH8A7,5,79,0,Telangana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229997,229998,29370,A1YFK6EHSVJKNH,5,51,0,Punjab,Starsy Printed Women's Round Neck White T-Shirt,Starsy


In [6]:
df1 = df.dropna()

In [7]:
sample_size = 5000  # Adjust as needed
top_k_features = 500  # Adjust as needed

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)


In [8]:
df2

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand
83902,83903,10353,AQGK8C8DG13JJ,4,78,0,Uttarakhand,Magic 22K Yellow Gold Plated Brass Chain,Magic
24030,24031,2986,A2KQY2NLA4N9X6,5,9,1,Andhra Pradesh,C.E Collection Polyester Orange Plain Eyelet D...,C.E Collection
187099,187100,23828,A1RH54KDESZ72N,4,57,0,Tripura,Allure Auto CM 1523 Car Mat Skoda Fabia,Allure Auto
92312,92313,11598,A38GMSNDDOXRCK,5,69,1,West Bengal,Durian Tulsa Leatherette 3 Seater Sofa,Durian
82973,82974,10291,A2B9EOVVB5XYN4,2,49,0,Assam,Lolprint Rakhi Gifts Scroll Greeting Card,Lolprint
...,...,...,...,...,...,...,...,...,...
192646,192647,24542,A3B4OWZ3D58K8I,5,57,0,Uttarakhand,"Voylla Metal, Alloy Necklace",Voylla
66747,66748,8341,A1JGZZFJVNDUW9,1,67,1,Haryana,First Smart Deal Plant Container Set,First Smart Deal
109221,109222,13794,A15IIJASHRF9IB,5,23,0,Goa,United Colors of Benetton Printed Round Neck T...,United Colors of Benetton
103234,103235,12997,A16Z8PH0P5RIQN,5,25,0,Uttar Pradesh,Karatcraft Twilite Gold Diamond 18 K Ring,Karatcraft


In [9]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load OrderHistory and ProductsList.csv files
order_history = pd.read_csv("OrderHistory.csv")  # Adjust the file path and options
products_list = pd.read_csv("ProductsList.csv")  # Adjust the file path and options

# Assuming you have columns 'ProductId' and 'product_description' in ProductsList.csv
# Map product descriptions to ProductId
product_descriptions = products_list.set_index('ProductId')['description'].to_dict()

# Load OrderHistory.csv and process it as needed
# ...

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)
df2.index = range(1, len(df2) + 1)
df2



,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand
1,83903,10353,AQGK8C8DG13JJ,4,78,0,Uttarakhand,Magic 22K Yellow Gold Plated Brass Chain,Magic
2,24031,2986,A2KQY2NLA4N9X6,5,9,1,Andhra Pradesh,C.E Collection Polyester Orange Plain Eyelet D...,C.E Collection
3,187100,23828,A1RH54KDESZ72N,4,57,0,Tripura,Allure Auto CM 1523 Car Mat Skoda Fabia,Allure Auto
4,92313,11598,A38GMSNDDOXRCK,5,69,1,West Bengal,Durian Tulsa Leatherette 3 Seater Sofa,Durian
5,82974,10291,A2B9EOVVB5XYN4,2,49,0,Assam,Lolprint Rakhi Gifts Scroll Greeting Card,Lolprint
...,...,...,...,...,...,...,...,...,...
4996,192647,24542,A3B4OWZ3D58K8I,5,57,0,Uttarakhand,"Voylla Metal, Alloy Necklace",Voylla
4997,66748,8341,A1JGZZFJVNDUW9,1,67,1,Haryana,First Smart Deal Plant Container Set,First Smart Deal
4998,109222,13794,A15IIJASHRF9IB,5,23,0,Goa,United Colors of Benetton Printed Round Neck T...,United Colors of Benetton
4999,103235,12997,A16Z8PH0P5RIQN,5,25,0,Uttar Pradesh,Karatcraft Twilite Gold Diamond 18 K Ring,Karatcraft


In [10]:
df2 = df1.sample(n=sample_size, random_state=42)
df2.index = range(1, len(df2) + 1)

# Merge df2 with product_descriptions based on ProductId
df2 = df2.merge(products_list[['ProductId', 'description']], on='ProductId', how='left')

# Fill missing descriptions with an empty string
df2['description'] = df2['description'].fillna('')
df2



,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,description
0,83903,10353,AQGK8C8DG13JJ,4,78,0,Uttarakhand,Magic 22K Yellow Gold Plated Brass Chain,Magic,Key Features of Magic 22K Yellow Gold Plated B...
1,24031,2986,A2KQY2NLA4N9X6,5,9,1,Andhra Pradesh,C.E Collection Polyester Orange Plain Eyelet D...,C.E Collection,Key Features of C.E Collection Polyester Multi...
2,187100,23828,A1RH54KDESZ72N,4,57,0,Tripura,Allure Auto CM 1523 Car Mat Skoda Fabia,Allure Auto,
3,92313,11598,A38GMSNDDOXRCK,5,69,1,West Bengal,Durian Tulsa Leatherette 3 Seater Sofa,Durian,Eira Casual Printed Women's Kurti - Buy Blue E...
4,82974,10291,A2B9EOVVB5XYN4,2,49,0,Assam,Lolprint Rakhi Gifts Scroll Greeting Card,Lolprint,Specifications of DWM Cotton Solid Patiala Gen...
...,...,...,...,...,...,...,...,...,...,...
4995,192647,24542,A3B4OWZ3D58K8I,5,57,0,Uttarakhand,"Voylla Metal, Alloy Necklace",Voylla,
4996,66748,8341,A1JGZZFJVNDUW9,1,67,1,Haryana,First Smart Deal Plant Container Set,First Smart Deal,Buy Color Palatte Plant Container Set for Rs.4...
4997,109222,13794,A15IIJASHRF9IB,5,23,0,Goa,United Colors of Benetton Printed Round Neck T...,United Colors of Benetton,Key Features of GALLOWAY skinny Fit Women's Je...
4998,103235,12997,A16Z8PH0P5RIQN,5,25,0,Uttar Pradesh,Karatcraft Twilite Gold Diamond 18 K Ring,Karatcraft,"Enthopia Back Cover for iPad Mini, iPad Mini 2..."


In [11]:
tfidf = TfidfVectorizer(stop_words='english')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['description'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()


In [12]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['description'].iloc[movie_indices]

# Example usage
recommended_products = get_recommendations(8034)
recommended_products

1770    Key Features of Famaya Girl's Leggings Black, ...
4538    Key Features of Avyak Women's White Leggings W...
2436    Generation Z Women's Leggings (Pack of 3)\n   ...
550     NE Women's Leggings - Buy Multicolor NE Women'...
395     Indiweaves Women's Leggings - Buy Purple, Blue...
2497    Indiweaves Women's Leggings - Buy Purple, Blue...
3656    Indiweaves Women's Leggings - Buy Purple, Blue...
370     NE Women's Leggings - Buy Orange-Light Green-B...
419     NE Women's Leggings - Buy Orange-Light Green-B...
2523    NE Women's Leggings - Buy Orange-Light Green-B...
Name: description, dtype: object

In [23]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

app = Flask(__name__)

# Load OrderHistory and ProductsList.csv files
order_history = pd.read_csv("OrderHistory.csv")
products_list = pd.read_csv("ProductsList.csv")

# Assuming you have columns 'ProductId' and 'product_description' in ProductsList.csv
# Map product descriptions to ProductId
product_descriptions = products_list.set_index('ProductId')['description'].to_dict()

# Sample a subset of your data (assuming df1 is your DataFrame)
sample_size = 1000  # Adjust as needed
df2 = order_history.sample(n=sample_size, random_state=42)
df2.index = range(1, len(df2) + 1)

# Merge df2 with product_descriptions based on ProductId
df2 = df2.merge(products_list[['ProductId', 'description']], on='ProductId', how='left')

# Fill missing descriptions with an empty string
df2['description'] = df2['description'].fillna('')

# Create a TfidfVectorizer for product descriptions
tfidf = TfidfVectorizer(stop_words='english')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['description'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['ProductId'].iloc[movie_indices].tolist()

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    product_id = data.get('product_id')
    recommendations = get_recommendations(product_id)
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(debug=True,port = 9005)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:9005
Press CTRL+C to quit
 * Restarting with stat
/home/codespace/.local/lib/python3.10/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/codespace/.local/lib/python3.10/site-packages/traitlets/traitlets.py:2499: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use '76b46f28-9d48-4aea-861a-fe5cc7609ab1' instead of 'b"76b46f28-9d48-4aea-861a-fe5cc7609ab1"'.
  warn(
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/codespace/.python/current/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, i

SystemExit: 1

/home/codespace/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 